In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import matplotlib.pyplot as plt
import numpy as np

/Users/kjartan/opt/anaconda3/envs/pmlEnv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# A1

In [2]:
cuda = torch.cuda.is_available()
batch_size = 128
log_interval = 10
num_epochs = 10

torch.manual_seed(1) # args.seed

device = torch.device("cuda" if cuda else "cpu") # args.cuda
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {} # args.cuda

# Get train and test data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)



In [3]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD # -ELBO


def train(epoch, model, optimizer):
    model.train() # so that everything has gradients and we can do backprop and so on...
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad() # "reset" gradients to 0 for text iteration
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward() # calc gradients
        train_loss += loss.item()
        optimizer.step() # backpropagation

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch, model):
    model.eval()
    test_loss = 0
    with torch.no_grad(): # no_grad turns of gradients...
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
from Models import VAE, convVAE

model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"trainable parameters: {params:,}")

for epoch in range(1, num_epochs + 1):
    train(epoch, model, optimizer)
    #torch.save(model.state_dict(), "weights/VAE.pth")
    test(epoch, model)
    with torch.no_grad():
        sample = torch.randn(64, 2).to(device) # 20 -> 2
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/VAE/sample_' + str(epoch) + '.png')

In [ ]:
z = 2
model = convVAE(out_channels = 64, kernel_size = 4, hiddenDim=z).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"trainable parameters: {params:,}")

In [ ]:
for epoch in range(1, num_epochs + 1):
    train(epoch, model)
    #torch.save(model.state_dict(), "weights/convVAE.pth")
    test(epoch, model)
    with torch.no_grad():
        sample = torch.randn(64, z).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/convVAE/sample_' + str(epoch) + '.png')

# A2


In [ ]:
from Models import denoisingDiffusion as Diffusion
from Models import deNoise

eps_model = deNoise()
eps_model.to(device)
diff = Diffusion(eps_model, device=device)
optimizer = optim.Adam(eps_model.parameters(), lr=1e-3)
eps_model.train() # so that everything has gradients and we can do backprop and so on...
train_loss = 0
for epoch in range(1, num_epochs):
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad() # "reset" gradients to 0 for text iteration
        
        loss = diff.loss(data, None)
        loss.backward() # calc gradients
        train_loss += loss.item()
        optimizer.step() # backpropagation

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))
    if False:
        with torch.no_grad():
            sample = torch.randn(64, 2).to(device)
            t = torch.arange(diff.T, 0, step=-1, device=device)
            sample = diff.sample_p(sample, t)
            save_image(sample.view(64, 1, 28, 28).cpu(),
                        'results/diffusion/sample_' + str(epoch) + '.png')

In [ ]:
with torch.no_grad():
    samples = diff.sample_p_loop(data.shape)
    save_image(sample.view(64, 1, 28, 28).cpu(),
                'results/diffusion/sample_' + str(epoch) + '.png')

In [ ]:
from PPCA import PPCA
ppca = PPCA(train_loader.dataset, 30)
samples = ppca.plot()


In [7]:
from Models import BayesianVAE

prior = torch.distributions.Normal(torch.tensor([0]), torch.tensor([1]))
model = BayesianVAE(prior)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model.train() # so that everything has gradients and we can do backprop and so on...
train_loss = 0
for batch_idx, (data, _) in enumerate(train_loader):
    data = data.to(device)
    optimizer.zero_grad() # "reset" gradients to 0 for text iteration
    recon_batch, mu, logvar = model(data)
    print(recon_batch)
    loss = loss_function(recon_batch, data, mu, logvar)
    loss.backward() # calc gradients
    train_loss += loss.item()
    optimizer.step() # backpropagation

print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))



model.eval()
test_loss = 0
with torch.no_grad(): # no_grad turns of gradients...
    for i, (data, _) in enumerate(test_loader):
        data = data.to(device)
        recon_batch, mu, logvar = model(data)
        test_loss += model.KL_loss().item()

test_loss /= len(test_loader.dataset)
print('====> Test set loss: {:.4f}'.format(test_loss))

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<SigmoidBackward0>)


RuntimeError: all elements of input should be between 0 and 1